In [1]:
# Set up GRASS environment
import os
import csv
from IPython.display import Image
from helper import show_interactively, show
import grass.script as gs
# had to add shell = True to helper.py for this to work
from helper import initialize_GRASS_notebook
initialize_GRASS_notebook(binary='grass83',
                          grassdata="D:\\grassdata", location="FUTURES_zoning_prelim",
                          mapset="input")

In [2]:
import validation

In [27]:
# change to mapset to where appropriate results are (core, sub, or base)
# !g.mapset mapset=core
# !g.mapset mapset=sub

!g.mapset mapset=base
# !g.region vector=central_pines_regional_council@input res=30
# !r.mask raster=sa_counties@input

In [18]:
# check if mask is in place
!g.list mapset=core type=raster pattern="MASK"

MASK


In [9]:
%%writefile reclass.txt
-1 0 = 0
1 thru 100 = 1
* = 0

Writing reclass.txt


In [ ]:
# calculate probabilty rasters

In [ ]:
# reclass each year (step) to a binary
for step in range(1,11):
  for seed in range(1, 51):
      gs.run_command(
          "r.reclass",
          input=f"step_run{seed}_{step}",
          output=f"step_run{seed}_{step}_binary",
          rules="reclass.txt",
      )
  # compute probabilty of development for each year
  gs.run_command(
      "r.series",
      input=[f"step_run{seed}_{step}_binary" for seed in range(1, 51)],
      output="probability",
      method="sum",
      weights=[0.1] * 10,
      nprocs=2,
  )

In [22]:
# to get hits, misses, etc.
import json
metrics_base = []
n = 50
# !r.mask masking_2019  --o
for i in range(1, n + 1):
    !r.reclass input=final_run{i} output=final_run{i}_reclass rules=reclass.txt --o
    cats, results = validation.compute('urban_2021@input', f'final_run{i}_reclass', 'urban_2001@input')
    metrics_base.append(validation.print_results('json', cats, **results))
# 

In [37]:
# !g.mapset mapset=core
# !g.mapset mapset=sub
# !g.mapset mapset=base

Mapset switched. Your shell continues to use the history for the old mapset


In [38]:
# computing probabilty for final year (2021) 
# doesn't work with nprocs
# removed ,weights=[0.02] * 50,
gs.run_command(
    "r.series",
    input=[f"final_run{seed}_reclass" for seed in range(1, 51)],
    output="probability",
    method="sum"
)

In [ ]:
# validation.compute replaces gs.read_command('r.futures.validation', simulated=f'final_run{i}_reclass', reference='urban_2021@input', original='urban_2001@input', format='json'))

In [15]:
# save metrics to pickle file
import pickle
with open("metrics", "wb") as pf:
  pickle.dump(metrics, pf)

In [21]:
# save metrics to pickle file
import pickle
with open("metrics_sub", "wb") as pf:
  pickle.dump(metrics_sub, pf)

In [23]:
# save metrics to pickle file
import pickle
with open("metrics_base", "wb") as pf:
  pickle.dump(metrics_base, pf)

In [ ]:
# get info about developed cells per zone

In [ ]:
# can do for each simulation or just for the >50% probability?